In [1]:
import pandas as pd


financial = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/학습데이터/financial_train.xlsx")
financial
non_financial = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/학습데이터/non_financial_train.xlsx")
non_financial
train = pd.concat([financial.drop('레이블', axis=1), non_financial.drop('종목코드', axis=1)], axis=1)
test = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/테스트데이터/test.xlsx")
# Feature and label separation
X_train = train.drop('레이블',axis=1)
y_train = train['레이블']
X_test = test.drop('레이블',axis=1)
y_test = test['레이블']

In [36]:
test.columns

Index(['종목코드', '총자산', '총자본회전율', '부채비율', '매출액증가율', '자기자본회전율', '당기순이익', '최대주주변경',
       '대표이사변경', '전환사채', '파산신청', '거래정지', '불성실공시법인', '레이블'],
      dtype='object')

In [40]:
X_train.columns

Index(['종목코드', '총자산', '총자본회전율', '부채비율', '매출액증가율', '자기자본회전율', '당기순이익', '최대주주변경',
       '대표이사변경', '전환사채', '파산신청', '거래정지', '불성실공시법인'],
      dtype='object')

In [108]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd


financial = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/학습데이터/financial_train.xlsx")
financial
non_financial = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/학습데이터/non_financial_train.xlsx")
non_financial
train = pd.concat([financial.drop('레이블', axis=1), non_financial.drop('종목코드', axis=1)], axis=1)
test = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/테스트데이터/test.xlsx")
# Feature and label separation
X_train = train.drop('레이블',axis=1)
y_train = train['레이블']
X_test = test.drop('레이블',axis=1)
y_test = test['레이블']


# Split the data into train and test sets

# Scale the features for models that are sensitive to feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models
models = {
#     'Logistic Regression':LogisticRegression(
#     penalty='l2',              # 규제 유형: 'l1', 'l2', 'elasticnet', 'none'
#     C=1,                     # 규제 강도: 값이 작을수록 강한 규제
#     solver='newton-cg',            # 최적화 알고리즘: 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'
#     max_iter=1000,              # 최대 반복 횟수
#     random_state=40            # 난수 시드
# ),
    'Random Forest':RandomForestClassifier(
    # # n_estimators=100,           # 트리의 개수
    # # max_depth=None,             # 트리의 최대 깊이
    # min_samples_split=2,        # 노드를 분할하기 위한 최소 샘플 수
    # min_samples_leaf=1,         # 리프 노드가 되기 위한 최소 샘플 수
    # #max_features='auto',        # 각 트리에서 고려할 최대 피처 수
    # bootstrap=True,             # 부트스트랩 샘플링 여부
    random_state=40           # 난수 시드
),
    'Gradient Boosting': GradientBoostingClassifier(
    # # learning_rate=0.1,           # 학습률
    # # n_estimators=100,            # 부스팅 단계의 개수
    # max_depth=3,                 # 개별 결정 트리의 최대 깊이
    # min_samples_split=2,         # 노드를 분할하기 위한 최소 샘플 수
    # min_samples_leaf=1,          # 리프 노드가 되기 위한 최소 샘플 수
    # subsample=1.0,               # 각 트리에 사용할 샘플의 비율
    # max_features=None,           # 각 트리에서 고려할 최대 피처 수
    random_state=40              # 난수 시드
),
    'SVM': SVC(
    # C=1.0,                      # 규제 강도
    # kernel='rbf',               # 커널 종류: 'linear', 'poly', 'rbf', 'sigmoid'
    # degree=3,                   # 다항 커널일 때의 차수
    # gamma='scale',              # 'scale', 'auto' 또는 float 값
    # coef0=0.0,                  # 다항 및 시그모이드 커널의 독립 항
    # probability=False,          # 확률 추정 수행 여부
    random_state=40           # 난수 시드
),
    'Neural Network': MLPClassifier(
    # hidden_layer_sizes=(100,),   # 은닉층의 뉴런 개수와 은닉층 개수
    # activation='relu',           # 활성화 함수: 'identity', 'logistic', 'tanh', 'relu'
    # # solver='adam',               # 최적화 알고리즘: 'lbfgs', 'sgd', 'adam'
    # alpha=0.001,                # L2 규제 매개변수
    # batch_size='auto',           # 배치 크기
    # learning_rate='constant',    # 학습률 일정 방식: 'constant', 'invscaling', 'adaptive'
    max_iter=1000,                # 최대 반복 횟수
    random_state=40              # 난수 시드
)
}


# 선택된 피처 출력

# Evaluate models using cross-validation and display the results
results = {}
for name, model in models.items():
    # Use scaled data for SVM and Neural Network
    if name in ['SVM', 'Neural Network']:
        sfs = SequentialFeatureSelector(model, n_features_to_select=5, direction='forward', cv=5)
        sfs.fit(X_train_scaled, y_train)
        selected_features = X_train.columns[sfs.get_support()]
        print(name,selected_features)
        model.fit(X_train_scaled[:,sfs.get_support()], y_train)
        y_pred = model.predict(X_test_scaled[:,sfs.get_support()])
        #print(selected_features)
    else:
        sfs = SequentialFeatureSelector(model, n_features_to_select=5, direction='forward', cv=5)
        sfs.fit(X_train, y_train)
        selected_features = X_train.columns[sfs.get_support()]
        print(name,selected_features)
        model.fit(X_train[selected_features], y_train)
        y_pred = model.predict(X_test[selected_features])
     
    # Calculate the metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(name,'\n',classification_report(y_test, y_pred, digits=4))
    l=[]
    for i in range(3):
        l.append(float(classification_report(y_test, y_pred, digits=4).split('\n')[-3].split('     ')[1].split('    ')[i]))
    sum(l)/3
    # Store the results
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
        
    }

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
results_df.sort_values(by='Accuracy', ascending=False, inplace=True)



Random Forest Index(['최대주주변경', '대표이사변경', '파산신청', '거래정지', '불성실공시법인'], dtype='object')
Random Forest 
               precision    recall  f1-score   support

           0     1.0000    0.9737    0.9867        76
           1     0.9231    1.0000    0.9600        24

    accuracy                         0.9800       100
   macro avg     0.9615    0.9868    0.9733       100
weighted avg     0.9815    0.9800    0.9803       100

Gradient Boosting Index(['총자산', '총자본회전율', '매출액증가율', '파산신청', '거래정지'], dtype='object')
Gradient Boosting 
               precision    recall  f1-score   support

           0     0.9737    0.9737    0.9737        76
           1     0.9167    0.9167    0.9167        24

    accuracy                         0.9600       100
   macro avg     0.9452    0.9452    0.9452       100
weighted avg     0.9600    0.9600    0.9600       100

SVM Index(['부채비율', '자기자본회전율', '대표이사변경', '파산신청', '거래정지'], dtype='object')
SVM 
               precision    recall  f1-score   support

      

In [133]:
l=[]
for i in range(3):
    l.append(float(classification_report(y_test, y_pred, digits=4).split('\n')[-3].split('     ')[1].split('    ')[i]))

## 0.9738666666666668 40



0.9617333333333334

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

best_best = 0.0
best_model_state = 0
for i in range(1,100):
    print(i)
    rft = RandomForestClassifier(random_state=i)
    rft.fit(X_train[['최대주주변경', '대표이사변경', '파산신청', '거래정지', '불성실공시법인']], y_train)
    y_pred = rft.predict(X_test[['최대주주변경', '대표이사변경', '파산신청', '거래정지', '불성실공시법인']])

    l=[]
    for j in range(3):
        l.append(float(classification_report(y_test, y_pred, digits=4).split('\n')[-3].split('     ')[1].split('    ')[j]))
    best = sum(l)/3
    print(best)
    if best >=best_best:
        best_best = best
        
        best_model_state = i
        print(best_best, best_model_state)


1
0.9738666666666668
0.9738666666666668 1
2
0.9738666666666668
0.9738666666666668 2
3
0.9738666666666668
0.9738666666666668 3
4
0.9738666666666668
0.9738666666666668 4
5
0.9617333333333334
6
0.9617333333333334
7
0.9617333333333334
8
0.9617333333333334
9
0.9617333333333334
10
0.9617333333333334
11
0.9738666666666668
0.9738666666666668 11
12
0.9617333333333334
13
0.9617333333333334
14
0.9617333333333334
15
0.9617333333333334
16
0.9617333333333334
17
0.9617333333333334
18
0.9617333333333334
19
0.9738666666666668
0.9738666666666668 19
20
0.9738666666666668
0.9738666666666668 20
21
0.9617333333333334
22
0.9617333333333334
23
0.9617333333333334
24
0.9738666666666668
0.9738666666666668 24
25
0.9738666666666668
0.9738666666666668 25
26
0.9617333333333334
27
0.9617333333333334
28
0.9617333333333334
29
0.9738666666666668
0.9738666666666668 29
30
0.9617333333333334
31
0.9738666666666668
0.9738666666666668 31
32
0.9617333333333334
33
0.9617333333333334
34
0.9617333333333334
35
0.9617333333333334
3

In [6]:
best_best

0.9738666666666668

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier

# 순차적 피처 선택기 정의
sfs = SequentialFeatureSelector(RandomForestClassifier(), n_features_to_select=5, direction='forward', cv=5)
sfs.fit(X, y)

# 선택된 피처 출력
selected_features = X.columns[sfs.get_support()]
print(selected_features)


In [45]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 데이터 불러오기
train_df = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/학습데이터/financial_train.xlsx")
test_df = pd.read_excel("C:/Users/humming/Downloads/상장폐지 종목 예측 프로젝트/테스트데이터/test.xlsx")

# 종목코드 열 제거 및 피처와 레이블 분리
X_train_data = train_df[['총자산', '총자본회전율', '부채비율', '매출액증가율', '자기자본회전율', '당기순이익']]
y_train_data = train_df['레이블']

X_test_data = test_df[['총자산', '총자본회전율', '부채비율', '매출액증가율', '자기자본회전율', '당기순이익']]
y_test_data = test_df['레이블']

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_data)
X_test_scaled = scaler.transform(X_test_data)

# 텐서로 변환
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1)  # 시퀀스 차원 추가
y_train_tensor = torch.tensor(y_train_data.values, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1)  # 시퀀스 차원 추가
y_test_tensor = torch.tensor(y_test_data.values, dtype=torch.float32).unsqueeze(1)

# 데이터셋과 데이터로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)


# RNN 모델 정의
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.relu(out[:, -1, :])
        out = self.fc(out)
        return torch.sigmoid(out)

# 하이퍼파라미터 설정
input_size = X_train_tensor.shape[2]  # input_size는 특성(feature)의 수
print(input_size,'ddd')
hidden_size = 86
output_size = 1
learning_rate = 0.01
num_epochs = 1000

# 모델 초기화, 손실 함수 및 옵티마이저 정의
model = RNNModel(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 최고 성능 기록을 위한 변수 초기화
best_f1 = 0.0000
best_model_path = 'best_model.pth'

# 모델 학습
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        # 순전파
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 테스트 데이터로 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        y_pred = []
        y_true = []
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch).squeeze()
            predicted = (outputs > 0.5).float()
            y_pred.extend(predicted.tolist())
            y_true.extend(y_batch.tolist())
            accuracy = accuracy_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
    # 현재 에포크에서의 모델이 최고 성능을 기록했다면 저장
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), best_model_path)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {f1*100:.2f}%')

# 최고 성능 모델 로드
best_model = RNNModel(input_size, hidden_size, output_size)
best_model.load_state_dict(torch.load(best_model_path, map_location=torch.device('cpu')))  # map_location 추가 가능
best_model.eval()


print(f'Best model accuracy: {best_f1* 100:.4f}%')

# 테스트 데이터에 대해 최종 예측 수행
with torch.no_grad():
    test_outputs = best_model(X_test_tensor)
    predicted = test_outputs.round()  # 최종 예측 값 반환


6 ddd
Epoch [1/1000], Loss: 0.7069, Accuracy: 8.00%
Epoch [2/1000], Loss: 0.4497, Accuracy: 15.38%
Epoch [3/1000], Loss: 0.3474, Accuracy: 57.14%
Epoch [4/1000], Loss: 0.2405, Accuracy: 61.11%
Epoch [5/1000], Loss: 0.4412, Accuracy: 71.79%
Epoch [6/1000], Loss: 0.1951, Accuracy: 71.79%
Epoch [7/1000], Loss: 0.1485, Accuracy: 70.00%
Epoch [8/1000], Loss: 0.5135, Accuracy: 73.91%
Epoch [9/1000], Loss: 0.1380, Accuracy: 74.42%
Epoch [10/1000], Loss: 0.5499, Accuracy: 73.17%
Epoch [11/1000], Loss: 0.2361, Accuracy: 78.26%
Epoch [12/1000], Loss: 0.3971, Accuracy: 71.79%
Epoch [13/1000], Loss: 0.1012, Accuracy: 73.91%
Epoch [14/1000], Loss: 0.1745, Accuracy: 73.17%
Epoch [15/1000], Loss: 0.0614, Accuracy: 74.42%
Epoch [16/1000], Loss: 0.4197, Accuracy: 72.73%
Epoch [17/1000], Loss: 0.1365, Accuracy: 71.43%
Epoch [18/1000], Loss: 0.3252, Accuracy: 78.26%
Epoch [19/1000], Loss: 0.5091, Accuracy: 72.73%
Epoch [20/1000], Loss: 0.1589, Accuracy: 75.00%
Epoch [21/1000], Loss: 0.4266, Accuracy: 74.

C:\Users\humming\AppData\Local\Temp\ipykernel_21196\3379596615.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path, m

In [41]:
print(f'Best model accuracy: {best_f1* 100:.4f}%')

Best model accuracy: 86.2745%


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
with torch.no_grad():
    y_pred = []
    y_true = []
    for X_batch, y_batch in test_loader:

        outputs = best_model(X_batch).squeeze()
        predicted = (outputs > 0.5).float()
        y_pred.extend(predicted.tolist())
        y_true.extend(y_batch.tolist())
        
# 성능 지표 계산
accuracy1 = accuracy_score(y_true, y_pred)
precision1 = precision_score(y_true, y_pred)
recall1 = recall_score(y_true, y_pred)
f11 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy1:.4f}')
print(f'Precision: {precision1:.4f}')
print(f'Recall: {recall1:.4f}')
print(f'F1 Score: {f11:.4f}')

Accuracy: 0.9400
Precision: 0.8750
Recall: 0.8750
F1 Score: 0.8750
